In [ ]:
from mpasanalysis import *
%matplotlib inline

In [ ]:
# flags
save_fig = False

In [ ]:
# get hostname
hostname = os.uname()[1]
print('Running on machine {}'.format(hostname))

In [ ]:
# set paths
if 'theta' in hostname:
    data_root = '/projects/ClimateEnergy_3/azamatm/E3SM_simulations/theta.20180906.branch_noCNT.A_WCYCL1950S_CMIP6_HR.ne120_oRRS18v3_ICG'
    fig_root = '/home/qingli/work/e3sm_res_cmp/figures/high_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/run/mpaso.hist.am.timeSeriesStatsMonthly.0051-01-01.nc'
elif 'edison' in hostname:
    data_root = '/global/cscratch1/sd/tang30/ACME_simulations/edison.20181204.noCNT.A_WCYCL1950S_CMIP6_LRtunedHR.ne30_oECv3_ICG'
    fig_root = '/global/homes/q/qingli/work/e3sm_res_cmp/figures/low_res'
    rst_file = data_root+'/run/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/archive/ocn/hist/mpaso.hist.am.timeSeriesStatsMonthly.0041-01-01.nc'
elif 'pn1803144' in hostname:
    # for testing
    data_root = os.environ['HOME']+'/data/mpas/test'
    fig_root = os.environ['HOME']+'/work/e3sm_res_cmp/figures/low_res'
    rst_file = data_root+'/mpaso.rst.0051-01-01_00000.nc'
    mon_file = data_root+'/mpaso.hist.am.timeSeriesStatsMonthly.0041-01-01.nc'
else:
    raise EnvironmentError('This script should be executed on edison, theta or pn1803144')
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# load dataset
f_rst = Dataset(rst_file, 'r')
f_mon = Dataset(mon_file, 'r')

In [ ]:
# read grid information
lon = np.degrees(f_rst.variables['lonCell'][:])
lat = np.degrees(f_rst.variables['latCell'][:])
cellArea = f_rst.variables['areaCell'][:]

refBottomDepth = f_rst.variables['refBottomDepth'][:]
nVertLevels = len(refBottomDepth)
refTopDepth = np.zeros(nVertLevels)
refTopDepth[1:nVertLevels] = refBottomDepth[0:nVertLevels-1]
refLayerThickness = refTopDepth-refBottomDepth
refMidDepth = 0.5*(refTopDepth+refBottomDepth)

In [ ]:
ncvar_temp = f_mon.variables['timeMonthly_avg_activeTracers_temperature']
temp = ncvar_temp[0,:,:]
print(temp.shape)

In [ ]:
mpaso_temp = MPASOVolume(data=temp, lon=lon, lat=lat, depth=refMidDepth, cellarea=cellArea,
                         name='Temperature', units='degC')
lon0 = 320
lat0 = 40
lon1 = 310
lat1 = 60
p = mpaso_temp.get_vertical_cross_section(lon0=lon0, lat0=lat0, lon1=lon1, lat1=lat1, depth_bottom=1000)

In [ ]:
p.plot(ptype='pcolor')

In [ ]:
mpaso_sst = MPASOMap(data=temp[:,0], lat=lat, lon=lon, cellarea=cellArea, name='SST', units='degC')
m = mpaso_sst.plot(region='LabSea')
x, y = m(p.lon, p.lat)
m.scatter(x, y, color='r',s=10, marker='.')
m.drawgreatcircle(lon0, lat0, lon1, lat1, color='y')
type(m)

In [ ]:
mpaso_temp.get_map(depth=500).plot(region='LabSea')